In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode
from nipype.interfaces import fsl
from nipype.interfaces.io import  SelectFiles, DataSink
from nipype.algorithms import modelgen, rapidart
from nipype.interfaces.base import Bunch

import nilearn

import os
import numpy as np

In [2]:
cont1 = ['Listening>Rest', 'T', ['listening'], [1.,0.,0.,0.,0.,0.,0.]]
cont2 = ['Rest>Listening', 'T', ['listening'], [-1.,0.,0.,0.,0.,0.,0.]]

glm_contrasts = [cont1,cont2]
# substitute the job info with your own paths
job_info = {"mni":"/usr/local/fsl/data/standard/MNI152_T1_2mm.nii.gz",  # substitute with your mni whole brain template
            "mni_brain":"/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz",# substitute with your mni extracted brain
            "data_dir": "/home/paradeisios/Documents/fsl_script/data/",
            "output_dir":"/home/paradeisios/Documents/fsl_script/output/",
            "subject_list":["1"],
            "n_scans":84,
            "TR":7,
            "glm_names":["listening"],
            "glm_onsets":[[42,126,210,294,378,462,546]],
            "glm_durations":[[42 for _ in range(7)]],
            "glm_contrasts" : glm_contrasts}


In [3]:
subject_info = Bunch(conditions=job_info["glm_names"],
                     onsets = job_info["glm_onsets"],
                     durations = job_info["glm_durations"])

In [4]:
##data grabber and data deposit


# Iterator - this loops though the subject your list provided in cell[2] - if you do not use standard bids, might need modification
infosource = Node(IdentityInterface(fields = ["subject_id"]),name="infosource")
infosource.iterables = [("subject_id", job_info["subject_list"])]


# String template with {}-based strings - this is the DATA GRABBER - you have to provide a template as to how the data are organized in the data folder, so it can find the anatomical and the funcitonal scans
template = { "anat": job_info["data_dir"]  + "sub-{subject_id}/anat/T1w.nii",
             "func": job_info["data_dir"]  + "sub-{subject_id}/func/T2w.nii"}

selectfiles = Node(SelectFiles(template),name='selectfiles')

# Data sink - this is the handler where your data will be output --- IF YOU DO NOT SPECIFY WHICH DATA YOU WANT TO BE SAVED, THEY WILL NOT BE SAVED (this is already done in the workflow, you can change it to add more or less saved outputs)

datasink = Node(DataSink(),name="datasink")
datasink.inputs.base_directory = job_info["output_dir"]

In [5]:
##structural workflow

bet = Node(fsl.BET(),name="bet")
bet.inputs.frac = 0.3 #value controlling how much of the skull to be removed - increasing might remove bits of brain
bet.inputs.vertical_gradient = 0.2 #value to linearly improve estimations at the bottom of the brain -increasing might understimate top
bet.inputs.mask = True #creates a binary mask of the brain


segment = Node(fsl.FAST(),name="segment")
"""You are interested mainly in the restored version of the structural and
    pve0 = grey matter
    pve1 = white matter
    pve2 = CSF """
segment.inputs.img_type = 1 # indicate its a T1 image
segment.inputs.number_classes = 3 # seperate structural into Grey Matter, White Matter and CSF
segment.inputs.output_biascorrected = True #return structural image bias field corrected
segment.inputs.output_biasfield = True #return bias field
segment.inputs.segments=True #create a binary mask for each tissue type
segment.inputs.no_pve=False #if you dont want partial volume estimation, set to true


flirt_struct = Node(fsl.FLIRT(),name="flirt_struct")
"""This step performs linear registration of the structural volume to the standard space"""
flirt_struct.inputs.reference = job_info["mni_brain"]
flirt_struct.inputs.dof = 12 #use 12 dof in structural flirt to prepare image for FNIRT nonlinear normalization to mni space
flirt_struct.inputs.out_matrix_file = "h2s_affine.mat" #keep this to facilite struct and func normalization to mni space using FNIRT
flirt_struct.inputs.interp = "spline" # change to trilinear if you are mostly interested in non-subcortical normalization. Trilinear is faster, but might have some rotational issues
flirt_struct.inputs.cost = "mutualinfo"


fnirt_struct = Node(fsl.FNIRT(),name="fnirt_struct")
"""This step performs non-linear registration of the structural volume to the standard space"""
fnirt_struct.inputs.ref_file = job_info["mni"]
fnirt_struct.inputs.config_file = "T1_2_MNI152_2mm"
fnirt_struct.inputs.fieldcoeff_file = True  #keep the deformation fields to faciliate func normalization with FNIRT

In [6]:
##functional workflow
img_to_float = Node(fsl.ImageMaths(),name="img_to_float")
"""This converts the 4d in float representation to faciliate some further computations"""
img_to_float.inputs.out_data_type='float'
img_to_float.inputs.op_string=''


vol_removal = Node(fsl.ExtractROI(), name="vol_removal")
"""This step removes the 5 first volumes to avoid T1 saturation"""
vol_removal.inputs.t_min = 0 #number of scans to exclude from functional image
vol_removal.inputs.t_size = -1 # keep all the rest until the end


realign = Node(fsl.MCFLIRT(),name = "realign")
"""This step realigns data to the first image"""
realign.inputs.save_mats = True #save realignment  parameters
realign.inputs.save_plots = True #save parameter plots
realign.inputs.dof = 6 #use this for rigid body correction


art = Node(rapidart.ArtifactDetect(),name="art")
"""This step performs artifact detection"""
art.inputs.mask_type = "spm_global"
art.inputs.parameter_source = 'FSL'
art.inputs.norm_threshold = 1
art.inputs.use_differences = [True, False]
art.inputs.zintensity_threshold = 3


mean_img = Node(fsl.maths.MeanImage(),name= "mean_img")
mean_img.inputs.dimension = "T" # find mean image across the time dimension


flirt_func = Node(fsl.FLIRT(),name = "flirt_func")
"""This step performs linear registration of the functional volume to the structural volume. It is mainly used to
extract the transformation matrix to later register to standard space"""
flirt_func.inputs.dof = 6
flirt_func.inputs.out_matrix_file = "f2h_affine.mat"
flirt_func.inputs.interp = "spline"
flirt_func.inputs.cost = "mutualinfo"


func_warp = Node(fsl.ApplyWarp(),name = "func_warp")
"""This step performs non linear registration to standard space using the affine matrix extracted
in the flirt above and the fnirt non linear warp in the structural pipeline """
func_warp.inputs.ref_file = job_info["mni"]


smooth = Node(fsl.IsotropicSmooth(),name="smooth")
"""This step performs smoothing"""
smooth.inputs.fwhm = 6 #change the smoothing kernel

In [7]:
fslmodel = Node(modelgen.SpecifyModel(),name="fslmodel") #gather all the information
fslmodel.inputs.subject_info=subject_info
fslmodel.inputs.high_pass_filter_cutoff=128.
fslmodel.inputs.time_repetition=job_info["TR"]
fslmodel.inputs.input_units="secs"

level1design = Node(fsl.Level1Design(), name="level1design")# make the model
level1design.inputs.bases = {'dgamma':{'derivs': False}}
level1design.inputs.interscan_interval = 7.
level1design.inputs.model_serial_correlations=True
level1design.inputs.contrasts=job_info["glm_contrasts"]

modelgen = Node(fsl.FEATModel(),name='modelgen') # transform to fsl model

modelestimate = Node(fsl.FILMGLS(), name='modelestimate') #run the model
modelestimate.inputs.smooth_autocorr=True 

In [8]:
struct_flow = Workflow(name="struct_flow", base_dir=job_info["output_dir"])
struct_flow.connect(bet,"out_file",segment,"in_files")
struct_flow.connect(segment,"restored_image",flirt_struct,"in_file")
struct_flow.connect(flirt_struct,"out_matrix_file",fnirt_struct,"affine_file")

#########################################################################################################
func_flow = Workflow(name="func_flow", base_dir=job_info["output_dir"])
func_flow.connect(img_to_float,"out_file",vol_removal,"in_file")
func_flow.connect(vol_removal, "roi_file", realign, "in_file")
func_flow.connect(realign,"out_file",art,"realigned_files")
func_flow.connect(realign,"par_file",art,"realignment_parameters")
func_flow.connect(realign,"out_file",mean_img,"in_file")
func_flow.connect(mean_img,"out_file",flirt_func,"in_file")
func_flow.connect(flirt_func,"out_matrix_file",func_warp,"premat")
func_flow.connect(realign,"out_file",func_warp,"in_file")
func_flow.connect(func_warp,"out_file",smooth,"in_file")

#######################################################################################################

first_level_flow = Workflow(name="first_flow",base_dir=job_info["output_dir"])

first_level_flow.connect(fslmodel,"session_info",level1design,"session_info")
first_level_flow.connect(level1design,"fsf_files",modelgen,"fsf_file")
first_level_flow.connect(level1design,"ev_files",modelgen,"ev_files")
first_level_flow.connect(modelgen,"design_file",modelestimate,"design_file")
first_level_flow.connect(modelgen,"con_file",modelestimate,"tcon_file")

########################################################################################################

analysis_flow = Workflow(name="analysis_flow",base_dir=job_info["output_dir"])

analysis_flow.connect([(infosource,selectfiles,[("subject_id","subject_id")]),
                         (selectfiles,struct_flow,[("anat","bet.in_file")]),
                         (selectfiles,struct_flow,[("anat","fnirt_struct.in_file")]),
                         (selectfiles,func_flow,[("func","img_to_float.in_file")]),
                         (struct_flow,func_flow,[("segment.restored_image","flirt_func.reference")]),
                         (struct_flow,func_flow,[("fnirt_struct.fieldcoeff_file","func_warp.field_file")]),
                         (func_flow,first_level_flow,[("smooth.out_file","fslmodel.functional_runs")]),
                         (func_flow,first_level_flow,[("realign.par_file","fslmodel.realignment_parameters")]),
                         (func_flow,first_level_flow,[("smooth.out_file","modelestimate.in_file")])])

#######################################################################################################

#this is your datasink, where you stash the results you want

analysis_flow.connect([(infosource,datasink,[("subject_id", "container")]),
                         #########################################################################
                         (struct_flow,datasink,[("bet.out_file","bet.@bet_brain")]),
                         (struct_flow,datasink,[("bet.mask_file","bet.@bet_mask")]),
                         (struct_flow,datasink,[("segment.restored_image","segment.@restored_image")]),
                         (struct_flow,datasink,[("segment.tissue_class_files","segment.@tissue_class_files")]),
                         (struct_flow,datasink,[("segment.bias_field","segment.@bias_field")]),
                         (struct_flow,datasink,[("segment.tissue_class_map","segment.@tissue_class_map")]),
                         (struct_flow,datasink,[("flirt_struct.out_file","flirt_struct.@structural_flirt")]),
                         (struct_flow,datasink,[("flirt_struct.out_matrix_file","flirt_struct.@affine_parameters")]),
                         (struct_flow,datasink,[("fnirt_struct.warped_file","flirt_struct.@structural_fnirt")]),
                         (struct_flow,datasink,[("fnirt_struct.field_file","flirt_struct.@fieldcoeff_file")]),

                         #########################################################################
                         (func_flow,datasink,[("vol_removal.roi_file","vol_removal.@files")]),
                         (func_flow,datasink,[("realign.out_file","realign.@realigned_files")]),
                         (func_flow,datasink,[("realign.par_file","realign.@realig_parameters")]),
                         (func_flow,datasink,[("art.outlier_files","art.@outlier_files")]),
                         (func_flow,datasink,[("art.plot_files","art.@plot_files")]),
                         (func_flow,datasink,[("art.displacement_files","art.@displacement_files")]),
                         (func_flow,datasink,[("mean_img.out_file","mean_img.@mean_img")]),
                         (func_flow,datasink,[("flirt_func.out_matrix_file","flirt_func.@affine_parameters")]),
                         (func_flow,datasink,[("flirt_func.out_file","flirt_func.@functional_flirt")]),
                         (func_flow,datasink,[("func_warp.out_file","func_warp.@functional_fnirt")]),
                         (func_flow,datasink,[("smooth.out_file","smooth.@smoothed_files")]),
                      
                         ##########################################################################
                         (first_level_flow,datasink,[("modelgen.design_image","model_design.@desmat")]),
                         (first_level_flow,datasink,[("modelgen.design_cov","model_design.@covariance")]),
                         (first_level_flow,datasink,[("modelestimate.copes","modelestimate.@copes")]),
                         (first_level_flow,datasink,[("modelestimate.varcopes","modelestimate.@varcopes")]),
                         (first_level_flow,datasink,[("modelestimate.zstats","modelestimate.@zstats")])])

In [9]:
analysis_flow.write_graph("workflow_graph.dot")

analysis_flow.write_graph(graph2use='colored')
analysis_flow.write_graph(graph2use='flat')

210514-18:53:52,805 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Documents/fsl_script/output/analysis_flow/workflow_graph.png (graph2use=hierarchical, simple_form=True).
210514-18:53:54,242 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Documents/fsl_script/output/analysis_flow/graph.png (graph2use=colored, simple_form=True).
210514-18:53:57,759 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Documents/fsl_script/output/analysis_flow/graph.png (graph2use=flat, simple_form=True).


'/home/paradeisios/Documents/fsl_script/output/analysis_flow/graph.png'

In [10]:
analysis_flow.run(plugin='MultiProc', plugin_args={'n_procs' : 4})

210514-18:53:57,798 nipype.workflow INFO:
	 Workflow analysis_flow settings: ['check', 'execution', 'logging', 'monitoring']
210514-18:53:57,824 nipype.workflow INFO:
	 Running in parallel.
210514-18:53:57,827 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 13.79/13.79, Free processors: 4/4.
210514-18:53:57,890 nipype.workflow INFO:
	 [Node] Setting-up "analysis_flow.selectfiles" in "/home/paradeisios/Documents/fsl_script/output/analysis_flow/_subject_id_1/selectfiles".
210514-18:53:57,895 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210514-18:53:57,901 nipype.workflow INFO:
	 [Node] Finished "analysis_flow.selectfiles".
210514-18:53:59,834 nipype.workflow INFO:
	 [Job 0] Completed (analysis_flow.selectfiles).
210514-18:53:59,845 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 13.79/13.79, Free processors: 4/4.
210514-18:53:59,933 nipype.workflow INFO:
	 [Job 1